# X-ray Absorption Spectroscopy (XAS) Experiment at SCS

Author: Jun Zhu jun.zhu@xfel.eu

Created on Nov. 12, 2018

Copyright (C) European X-Ray Free-Electron Laser Facility GmbH.

All rights reserved.

## Introduction

<table><tr>
    <td>
        <img src='misc/mono.png' width="600 px">
        <figcaption>Higley et al., Rev. Sci. Instrum.  87, 033110 (2016)</figcaption>
    </td>
    <td>
        <img src='misc/illustration.png' width="920 px">
    </td>
    <td>
        <img src='misc/expected_data.png' width="800 px">
        <figcaption>Higley et al., submitted</figcaption></td>
</tr></table>

## Data Analysis

In [ ]:
import os
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
try:
    # for interactive plot
    from plotly.offline import init_notebook_mode, iplot
    import plotly.graph_objs as go
    init_notebook_mode(connected=True)
    
    USE_PLOTLY = True
except ImportError:
    USE_PLOTLY = False

from xas_processor import XasDigitizer, compute_absorption

%load_ext autoreload
%autoreload 2

In [ ]:
!ls /gpfs/exfel/exp/SCS/201831/p900048/raw

```py
class XasDigitizer(XasProcessor):
     def __init__(self, run_folder, channels=('D', 'B', 'C', 'A'), 
                  pulse_separation=880e-9, interleaved_mode=False, pulse_id_min=0, n_pulses=1):
         """Initialization.
         
         :param str run_folder: full path of the run folder.
         :param tuple channels: names of AdqDigitizer channels which connects to MCP1 to MCP4.
         :param float pulse_separation: pulse separation in a train, in s.
         :param bool interleaved_mode: the resolution is improved by a factor of two 
             in the interleaved mode. Default = False.
         :param int pulse_id_min: start of the pulse ID.
         :param int n_pulses: number of pulses in a train.
         """
```

In [ ]:
run_folder = "/gpfs/exfel/exp/SCS/201831/p900048/raw/r0315"

# instantiate an `XasDigitizer` object
xas = XasDigitizer(run_folder, pulse_id_min=30, n_pulses=10)
xas.info()

## Sanity check

```py
def plot_xgm_run(self, *, figsize=(8, 5.6)):
    """Plot the train resolved data from XGM.
    
    :param tuple figsize: figure size.
    """
```

In [ ]:
fig, (ax11, ax12, ax2) = xas.plot_xgm_run()

why negative pulse energies?

```py
def plot_xgm_train(self, *, index=0, train_id=None, figsize=(8, 5.6)):
    """Plot xgm measurement in a given train.

    :param int index: train index. Ignored if train_id is given.
    :param int train_id: train ID.
    :param tuple figsize: figure size.
    """
```

In [ ]:
fig, ax = xas.plot_xgm_train(index=10)

```py
def plot_digitizer_train(self, *, index=0, train_id=None, figsize=(8, 11.2),
                         x_min=None, x_max=None):
    """Plot digitizer signals in a given train.

    :param int index: train index. Ignored if train_id is given.
    :param int train_id: train ID.
    :param tuple figsize: figure size.
    :param int x_min: minimum sample ID.
    :param int x_max: maximum sample ID.
    """
```

In [ ]:
fig, axes = xas.plot_digitizer_train(index=200, x_min=12000, x_max=38000)

## Correlation between I0 and I1

```py
def process(self, config=None):
    """Process the run data.

    :param dict config: configuration for integrating of digitizer signal.
    """
```

In [ ]:
import time

t0 = time.time()

config = {"sample_start": 14590, "peak_width": 30, "background_end": 14580, "background_width": 500}
xas.process(config)  # processing will take about 60 seconds for a 5-minute run

print("Processing time: {:.1f} seconds".format(time.time() - t0))

```py
 def get_dataframe(self):
     """Get the pulse-resolved data in pandas.DataFrame.

     :return pandas.DataFrame: pulse resolved data in pandas.DataFrame.
     """
```

In [ ]:
xas.get_dataframe().head()

In [ ]:
# get absorption data as pandas.DataFrame
xas.absorption

```py
def plot_correlation(self, channel="all", *, figsize=(8, 6), marker_size=6, alpha=0.05, n_bins=20):
    """Generate correlation plots.

    :param str channel: MCP channel name, e.g. mcp1, for visualizing
        a single channel with four plots, or 'all' for visualizing all 
        the channels with one plot each. Case insensitive.
    :param tuple figsize: figure size.
    :param int marker_size: marker size for the scatter plots.
    :param float alpha: transparency for the scatter plots.
    :param int n_bins: number of bins for the histogram plots.
    """
```

In [ ]:
fig, ax = xas.plot_correlation()

In [ ]:
fig, ax = xas.plot_correlation('mcp1')

In [ ]:
fig, ax = xas.plot_correlation('mcp2')

In [ ]:
fig, ax = xas.plot_correlation('mcp3')

In [ ]:
fig, ax = xas.plot_correlation('mcp4')

## Absorption spectrum

```py
def compute_spectrum(self, n_bins=20):
    """Calculate spectrum.

    :param int n_bins: number of energy bins.

    :return: spectrum data in pandas.DataFrame.
    """
```

In [ ]:
spectrum = xas.compute_spectrum()

spectrum